In [1]:
import glob
import ipywidgets
import ipyvolume
import cv2
import numpy as np
import os
import time

In [2]:
import pythreejs
figcam = pythreejs.PerspectiveCamera(aspect=0.8, fov=46.0, matrixWorldNeedsUpdate=True, position=(1.785528946985704, 1.3757523415625494, 2.412892214337929), projectionMatrix=(2.9448154572796907, 0.0, 0.0, 0.0, 0.0, 2.3558523658237527, 0.0, 0.0, 0.0, 0.0, -1.00010000500025, -1.0, 0.0, 0.0, -0.200010000500025, 0.0), quaternion=(-0.19796316984919116, 0.30892794387643674, 0.08046673488216134, 0.9267681551784899), rotation=(-0.5181948251143776, 0.5713287660544955, 0.3285944197012996, 'XYZ'), scale=(1.0, 1.0, 1.0), up=(-0.27146072492949946, 0.9086713759213286, -0.31721507751364897))

## Volume Rendering

In [3]:
levels = {

    'UCSD900':
         {  'SR': { '10': [0.10, 0.35], '15': [0.10, 0.35], '20': [0.10, 0.35], '25': [0.10, 0.35], '30': [0.10, 0.35], '40': [0.10, 0.35]},
            'LR': { '10': [0.20, 0.45], '15': [0.20, 0.60], '20': [0.20, 0.65], '25': [0.20, 0.75], '30': [0.20, 0.80], '40': [0.20, 0.80]},
            'HR': [1.00, 0.75]
         },
    'UCSD900flipped':
         {  'SR': { '10': [0.35, 0.10], '15': [0.35, 0.10], '20': [0.35, 0.10], '25': [0.35, 0.10], '30': [0.35, 0.10], '40': [0.35, 0.10]},
            'LR': { '10': [0.35, 0.20], '15': [0.57, 0.20], '20': [0.65, 0.20], '25': [0.73, 0.20], '30': [0.77, 0.20], '40': [0.77, 0.20]},
            'HR': [1.00, 0.75]
         },
    'S1':
         {  'SR': { '10': [0.35, 0.00], '15': [0.35, 0.00], '20': [0.35, 0.00], '25': [0.35, 0.00] },
            'LR': { '10': [0.50, 0.00], '15': [0.50, 0.00], '20': [0.50, 0.00], '25': [0.50, 0.00] },
            'HR': [1.00, 0.75]
         },
    'S2':
         {  'SR': { '10': [0.35, 0.00], '15': [0.35, 0.00], '20': [0.35, 0.00], '25': [0.35, 0.00] },
            'LR': { '10': [0.50, 0.00], '15': [0.50, 0.00], '20': [0.50, 0.00], '25': [0.50, 0.00] },
            'HR': [1.00, 0.75]
         },
    'S3':
         {  'SR': { '10': [0.35, 0.00], '15': [0.35, 0.00], '20': [0.35, 0.00], '25': [0.35, 0.00] },
            'LR': { '10': [0.50, 0.00], '15': [0.50, 0.00], '20': [0.50, 0.00], '25': [0.50, 0.00] },
            'HR': [1.00, 0.75]
         },
     'ucsdLED':
         {
             'SR': { '1k': [0.35, 0.00], '2k': [0.35, 0.00], '4k': [0.35, 0.00] },
             'LR': { '1k': [0.50, 0.00], '2k': [0.50, 0.00], '4k': [0.50, 0.00] },
             'HR': [1.00, 0.75]
         },
    'ucsdLEDsm':
         {
             'SR': { '1k': [0.35, 0.00], '2k': [0.35, 0.00], '4k': [0.35, 0.00] },
             'LR': { '1k': [0.50, 0.00], '2k': [0.50, 0.00], '4k': [0.50, 0.00] },
             'HR': [1.00, 0.75]
         }
}

opacities = {
     'UCSD900':  
            { 'SR': { '10': [0.00,0.20,0.00], '15': [0.00,0.20,0.00], '20': [0.00,0.20,0.00], '25': [0.00,0.20,0.00]},
              'LR': { '10': [0.00,0.20,0.00], '15': [0.00,0.20,0.00], '20': [0.00,0.20,0.00], '25': [0.00,0.20,0.00]},
              'HR': [0.70,0.73,0.00]
            },
     'UCSD900flipped':  
            { 'SR': { '10': [0.20,0.00,0.00], '15': [0.20,0.00,0.00], '20': [0.20,0.00,0.00], '25': [0.20,0.00,0.00], '30': [0.20,0.00,0.00], '40': [0.20,0.00,0.00]},
              'LR': { '10': [0.20,0.00,0.00], '15': [0.20,0.00,0.00], '20': [0.20,0.00,0.00], '25': [0.20,0.00,0.00], '30': [0.20,0.00,0.00], '40': [0.20,0.00,0.00]},
              'HR': [0.70,0.73,0.00]
            },
      'S1':
            { 'SR': { '10': [0.20,0.00,0.00], '15': [0.20,0.00,0.00], '20': [0.20,0.00,0.00], '25': [0.20,0.00,0.00]},
              'LR': { '10': [0.15,0.00,0.00], '15': [0.15,0.00,0.00], '20': [0.15,0.00,0.00], '25': [0.15,0.00,0.00]},
              'HR': [0.70,0.73,0.00]
            },
      'S2':
            { 'SR': { '10': [0.20,0.00,0.00], '15': [0.20,0.00,0.00], '20': [0.20,0.00,0.00], '25': [0.20,0.00,0.00]},
              'LR': { '10': [0.15,0.00,0.00], '15': [0.15,0.00,0.00], '20': [0.15,0.00,0.00], '25': [0.15,0.00,0.00]},
              'HR': [0.70,0.73,0.00]
            },
       'S3':
            { 'SR': { '10': [0.20,0.00,0.00], '15': [0.20,0.00,0.00], '20': [0.20,0.00,0.00], '25': [0.20,0.00,0.00]},
              'LR': { '10': [0.15,0.00,0.00], '15': [0.15,0.00,0.00], '20': [0.15,0.00,0.00], '25': [0.15,0.00,0.00]},
              'HR': [0.70,0.73,0.00]
            },
       'ucsdLED':
            { 'SR': { '1k': [0.20,0.00,0.00], '2k': [0.20,0.00,0.00], '4k': [0.20,0.00,0.00]},
              'LR': { '1k': [0.15,0.00,0.00], '2k': [0.15,0.00,0.00], '4k': [0.15,0.00,0.00]},
              'HR': [0.70,0.73,0.00]
            },
       'ucsdLEDsm':
            { 'SR': { '1k': [0.20,0.00,0.00], '2k': [0.20,0.00,0.00], '4k': [0.20,0.00,0.00]},
              'LR': { '1k': [0.15,0.00,0.00], '2k': [0.15,0.00,0.00], '4k': [0.15,0.00,0.00]},
              'HR': [0.70,0.73,0.00]
            },
}

In [4]:
fig_res = {} # dictionary of result figures
fig_inp = {} # dictionary of input figures

In [6]:
#noises = ['10','15','20','25']
#sample = 'ucsdLEDsm'
sample = "UCSD900flipped"
noises = list(levels[sample].keys())
#frame_range = [200,509,0,509]
frame_range = [0,90,0,390]
#frame_range = [0,512,0,512]
opacity = 5.00
brightness = 2.00

In [6]:
noise = '10'
input_vol = np.load('result_vol_npys/input_'+sample+'_vol_'+noise+'.npy')
input_vol = input_vol[:,frame_range[0]:frame_range[1],frame_range[2]:frame_range[3]]
mins = input_vol.min(axis=(0,1))
maxs = input_vol.max(axis=(0,1))
input_vol_norm = (input_vol - mins) / (maxs - mins)
fig_inp[noise] = ipyvolume.figure(camera = figcam)
ipyvolume.volshow(input_vol_norm[:,:,:],level=levels[sample]['LR'][noise],opacity=opacities[sample]['LR'][noise])
fig_inp[noise].volumes[0].opacity_scale = opacity
fig_inp[noise].volumes[0].brightness = brightness
ipyvolume.pylab.xyzlabel(' ', ' ', ' ')
ipyvolume.show()

/opt/conda/lib/python3.6/site-packages/ipyvolume/widgets.py:179: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  data_view = self.data_original[view]
/opt/conda/lib/python3.6/site-packages/ipyvolume/utils.py:204: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  data = (data[slices1] + data[slices2])/2
/opt/conda/lib/python3.6/site-packages/ipyvolume/serialize.py:81: RuntimeWarning: invalid value encountered in true_divide
  gradient = gradient / np.sqrt(gradient[0]**2 + gradient[1]**2 + gradient[2]**2)


In [7]:
ipyvolume.pylab.savefig('input_3dfig_'+sample+'_'+noise+'.png',width=1280,height=720,fig=fig_inp[noise])

Output()

In [8]:
avg_img = np.load('result_vol_npys/output_'+sample+'_vol_'+noise+'.npy')
avg_img = avg_img[:,frame_range[0]:frame_range[1],frame_range[2]:frame_range[3]]
mins = avg_img.min(axis=(0,1))
maxs = avg_img.max(axis=(0,1))
avg_img_norm = (avg_img - mins) / (maxs - mins)
fig_res[noise] = ipyvolume.figure(camera = figcam)
ipyvolume.volshow(avg_img_norm[:,:,:],level=levels[sample]['SR'][noise],opacity=opacities[sample]['SR'][noise])
#ipyvolume.selector_default()
fig_res[noise].volumes[0].opacity_scale = opacity
fig_res[noise].volumes[0].brightness = brightness
ipyvolume.pylab.xyzlabel(' ', ' ', ' ')
ipyvolume.show()

In [9]:
ipyvolume.pylab.savefig('output_3dfig_'+sample+'_'+noise+'.png',width=1280,height=720,fig=fig_res[noise])

Output()

In [10]:
noise = '15'
input_vol = np.load('result_vol_npys/input_'+sample+'_vol_'+noise+'.npy')
input_vol = input_vol[:,frame_range[0]:frame_range[1],frame_range[2]:frame_range[3]]
mins = input_vol.min(axis=(0,1))
maxs = input_vol.max(axis=(0,1))
input_vol_norm = (input_vol - mins) / (maxs - mins)
fig_inp[noise] = ipyvolume.figure(camera = figcam)
ipyvolume.volshow(input_vol_norm[:,:,:],level=levels[sample]['LR'][noise],opacity=opacities[sample]['LR'][noise])
fig_inp[noise].volumes[0].opacity_scale = opacity
fig_inp[noise].volumes[0].brightness = brightness
ipyvolume.pylab.xyzlabel(' ', ' ', ' ')
ipyvolume.show()

In [11]:
ipyvolume.pylab.savefig('input_3dfig_'+sample+'_'+noise+'.png',width=1280,height=720,fig=fig_inp[noise])

Output()

In [12]:
avg_img = np.load('result_vol_npys/output_'+sample+'_vol_'+noise+'.npy')
avg_img = avg_img[:,frame_range[0]:frame_range[1],frame_range[2]:frame_range[3]]
mins = avg_img.min(axis=(0,1))
maxs = avg_img.max(axis=(0,1))
avg_img_norm = (avg_img - mins) / (maxs - mins)
fig_res[noise] = ipyvolume.figure(camera = figcam)
ipyvolume.volshow(avg_img_norm[:,:,:],level=levels[sample]['SR'][noise],opacity=opacities[sample]['SR'][noise])
fig_res[noise].volumes[0].opacity_scale = opacity
fig_res[noise].volumes[0].brightness = brightness
ipyvolume.pylab.xyzlabel(' ', ' ', ' ')
ipyvolume.show()

In [13]:
ipyvolume.pylab.savefig('output_3dfig_'+sample+'_'+noise+'.png',width=1280,height=720,fig=fig_res[noise])

Output()

In [14]:
noise = '20'
input_vol = np.load('result_vol_npys/input_'+sample+'_vol_'+noise+'.npy')
input_vol = input_vol[:,frame_range[0]:frame_range[1],frame_range[2]:frame_range[3]]
mins = input_vol.min(axis=(0,1))
maxs = input_vol.max(axis=(0,1))
input_vol_norm = (input_vol - mins) / (maxs - mins)
fig_inp[noise] = ipyvolume.figure(camera = figcam)
ipyvolume.volshow(input_vol_norm[:,:,:],level=levels[sample]['LR'][noise],opacity=opacities[sample]['LR'][noise])
fig_inp[noise].volumes[0].opacity_scale = opacity
fig_inp[noise].volumes[0].brightness = brightness
ipyvolume.pylab.xyzlabel(' ', ' ', ' ')
ipyvolume.show()

In [15]:
ipyvolume.pylab.savefig('input_3dfig_'+sample+'_'+noise+'.png',width=1280,height=720,fig=fig_inp[noise])

Output()

In [16]:
avg_img = np.load('result_vol_npys/output_'+sample+'_vol_'+noise+'.npy')
avg_img = avg_img[:,frame_range[0]:frame_range[1],frame_range[2]:frame_range[3]]
mins = avg_img.min(axis=(0,1))
maxs = avg_img.max(axis=(0,1))
avg_img_norm = (avg_img - mins) / (maxs - mins)
fig_res[noise] = ipyvolume.figure(camera = figcam)
ipyvolume.volshow(avg_img_norm[:,:,:],level=levels[sample]['SR'][noise],opacity=opacities[sample]['SR'][noise])
fig_res[noise].volumes[0].opacity_scale = opacity
fig_res[noise].volumes[0].brightness = brightness
ipyvolume.pylab.xyzlabel(' ', ' ', ' ')
ipyvolume.show()

In [17]:
ipyvolume.pylab.savefig('output_3dfig_'+sample+'_'+noise+'.png',width=1280,height=720,fig=fig_res[noise])

Output()

In [18]:
noise = '25'
input_vol = np.load('result_vol_npys/input_'+sample+'_vol_'+noise+'.npy')
input_vol = input_vol[:,frame_range[0]:frame_range[1],frame_range[2]:frame_range[3]]
mins = input_vol.min(axis=(0,1))
maxs = input_vol.max(axis=(0,1))
input_vol_norm = (input_vol - mins) / (maxs - mins)
fig_inp[noise] = ipyvolume.figure(camera = figcam)
ipyvolume.volshow(input_vol_norm[:,:,:],level=levels[sample]['LR'][noise],opacity=opacities[sample]['LR'][noise])
fig_inp[noise].volumes[0].opacity_scale = opacity
fig_inp[noise].volumes[0].brightness = brightness
ipyvolume.pylab.xyzlabel(' ', ' ', ' ')
ipyvolume.show()

In [19]:
ipyvolume.pylab.savefig('input_3dfig_'+sample+'_'+noise+'.png',width=1280,height=720,fig=fig_inp[noise])

Output()

In [20]:
avg_img = np.load('result_vol_npys/output_'+sample+'_vol_'+noise+'.npy')
avg_img = avg_img[:,frame_range[0]:frame_range[1],frame_range[2]:frame_range[3]]
mins = avg_img.min(axis=(0,1))
maxs = avg_img.max(axis=(0,1))
avg_img_norm = (avg_img - mins) / (maxs - mins)
fig_res[noise] = ipyvolume.figure(camera = figcam,opacity_scale = 1.58)
ipyvolume.volshow(avg_img_norm[:,:,:],level=levels[sample]['SR'][noise],opacity=opacities[sample]['SR'][noise])
fig_res[noise].volumes[0].opacity_scale = opacity
fig_res[noise].volumes[0].brightness = brightness
ipyvolume.pylab.xyzlabel(' ', ' ', ' ')
ipyvolume.show()

In [22]:
ipyvolume.pylab.savefig('output_3dfig_'+sample+'_'+noise+'.png',width=1280,height=720,fig=fig_res[noise])

Output()

In [43]:
noise = '30'
input_vol = np.load('result_vol_npys/input_'+sample+'_vol_'+noise+'.npy')
input_vol = input_vol[:,frame_range[0]:frame_range[1],frame_range[2]:frame_range[3]]
mins = input_vol.min(axis=(0,1))
maxs = input_vol.max(axis=(0,1))
input_vol_norm = (input_vol - mins) / (maxs - mins)
fig_inp[noise] = ipyvolume.figure(camera = figcam)
ipyvolume.volshow(input_vol_norm[:,:,:],level=levels[sample]['LR'][noise],opacity=opacities[sample]['LR'][noise])
fig_inp[noise].volumes[0].opacity_scale = opacity
fig_inp[noise].volumes[0].brightness = brightness
ipyvolume.pylab.xyzlabel(' ', ' ', ' ')
ipyvolume.show()

In [44]:
ipyvolume.pylab.savefig('input_3dfig_'+sample+'_'+noise+'.png',width=1280,height=720,fig=fig_inp[noise])

Output()

In [45]:
avg_img = np.load('result_vol_npys/output_'+sample+'_vol_'+noise+'.npy')
avg_img = avg_img[:,frame_range[0]:frame_range[1],frame_range[2]:frame_range[3]]
mins = avg_img.min(axis=(0,1))
maxs = avg_img.max(axis=(0,1))
avg_img_norm = (avg_img - mins) / (maxs - mins)
fig_res[noise] = ipyvolume.figure(camera = figcam)
ipyvolume.volshow(avg_img_norm[:,:,:],level=levels[sample]['SR'][noise],opacity=opacities[sample]['SR'][noise])
fig_res[noise].volumes[0].opacity_scale = opacity
fig_res[noise].volumes[0].brightness = brightness
ipyvolume.pylab.xyzlabel(' ', ' ', ' ')
ipyvolume.show()

In [46]:
ipyvolume.pylab.savefig('output_3dfig_'+sample+'_'+noise+'.png',width=1280,height=720,fig=fig_res[noise])

Output()

In [47]:
noise = '40'
input_vol = np.load('result_vol_npys/input_'+sample+'_vol_'+noise+'.npy')
input_vol = input_vol[:,frame_range[0]:frame_range[1],frame_range[2]:frame_range[3]]
mins = input_vol.min(axis=(0,1))
maxs = input_vol.max(axis=(0,1))
input_vol_norm = (input_vol - mins) / (maxs - mins)
fig_inp[noise] = ipyvolume.figure(camera = figcam)
ipyvolume.volshow(input_vol_norm[:,:,:],level=levels[sample]['LR'][noise],opacity=opacities[sample]['LR'][noise])
fig_inp[noise].volumes[0].opacity_scale = opacity
fig_inp[noise].volumes[0].brightness = brightness
ipyvolume.pylab.xyzlabel(' ', ' ', ' ')
ipyvolume.show()

In [48]:
ipyvolume.pylab.savefig('input_3dfig_'+sample+'_'+noise+'.png',width=1280,height=720,fig=fig_inp[noise])

Output()

In [49]:
avg_img = np.load('result_vol_npys/output_'+sample+'_vol_'+noise+'.npy')
avg_img = avg_img[:,frame_range[0]:frame_range[1],frame_range[2]:frame_range[3]]
mins = avg_img.min(axis=(0,1))
maxs = avg_img.max(axis=(0,1))
avg_img_norm = (avg_img - mins) / (maxs - mins)
fig_res[noise] = ipyvolume.figure(camera = figcam)
ipyvolume.volshow(avg_img_norm[:,:,:],level=levels[sample]['SR'][noise],opacity=opacities[sample]['SR'][noise])
fig_res[noise].volumes[0].opacity_scale = opacity
fig_res[noise].volumes[0].brightness = brightness
ipyvolume.pylab.xyzlabel(' ', ' ', ' ')
ipyvolume.show()

In [50]:
ipyvolume.pylab.savefig('output_3dfig_'+sample+'_'+noise+'.png',width=1280,height=720,fig=fig_res[noise])

Output()

In [35]:
gt_vol = np.load('result_vol_npys/gt_'+sample+'_vol.npy')
gt_vol = gt_vol[:,frame_range[0]:frame_range[1],frame_range[2]:frame_range[3]]
mins = gt_vol.min(axis=(0,1))
maxs = gt_vol.max(axis=(0,1))
gt_vol_norm = (gt_vol - mins) / (maxs - mins)
fig_gt = ipyvolume.figure(camera = figcam)
ipyvolume.volshow(gt_vol_norm[:,:,:],level=levels[sample]['LR'][noise],opacity=opacities[sample]['LR'][noise],specular_exponent=5)
fig_gt.volumes[0].opacity_scale = opacity
fig_gt.volumes[0].brightness = brightness
ipyvolume.pylab.xyzlabel(' ', ' ', ' ')
ipyvolume.show()

In [36]:
ipyvolume.pylab.savefig('gt_3dfig_'+sample+'.png',width=1280,height=720,fig = fig_gt)

Output()

## Convert full UCSD sample data into MATLAB

In [40]:
import scipy.io as sio

In [42]:
noises = ['30','40']
for noise in noises:
    sr_data = np.load('result_vol_npys/output_'+sample+'_vol_'+noise+'.npy')
    lr_data = np.load('result_vol_npys/input_'+sample+'_vol_'+noise+'.npy')
    mins = sr_data.min(axis=(0,1))
    maxs = sr_data.max(axis=(0,1))
    sr_data_norm = (sr_data - mins) / (maxs - mins)
    mins = lr_data.min(axis=(0,1))
    maxs = lr_data.max(axis=(0,1))
    lr_data_norm = (lr_data - mins) / (maxs - mins)    
    sio.savemat('UCSD900_'+noise+'_LR.mat',{'vect':lr_data})
    sio.savemat('UCSD900_'+noise+'_SR.mat',{'vect':sr_data})
    sio.savemat('UCSD900_norm_'+noise+'_LR.mat',{'vect':lr_data_norm})
    sio.savemat('UCSD900_norm_'+noise+'_SR.mat',{'vect':sr_data_norm})
#hr_data = np.load('target_ucsd_vol.npy')
#mins = hr_data.min(axis=(0,1))
#maxs = hr_data.max(axis=(0,1))
#hr_data_norm = (hr_data - mins) / (maxs - mins) 
#sio.savemat('UCSD900_HR.mat',{'vect':hr_data})
#sio.savemat('UCSD900_norm_HR.mat',{'vect':hr_data_norm})